# Summary statistics data mungling in preparation for fine-mapping pipelines

This pipeline extracts loci of interest from association analysis summary statistics data, intersecting it with genotype data to compute correlation matrix, and output the data-set per loci with summary statistics and genotype correlations matched. Additionally it extracts prior inclusion probability (annotation scores) for each variant in the data-set, if the information is available.

This pipeline was devised by Gao Wang and implemented by Min Qiao at The University of Chicago.

## Input

- Summary statistics file, `gzip` compressed, including 7 columns
    - chromsome
    - position
    - reference allele
    - alternative allele
    - z-score
    - ${\beta}$ (effect size)
    - standard error (se)

  **Input have to be sorted by chromosome and position!**


- Loci file, similar in flavor to `bed` files.

      1st column is chr; 2nd is chunk start position; 3rd is chunk end position; 4th is loci identifier.

        chr22	44995308	46470495	1699
        chr22	46470495	47596318	1700
        chr22	47596318	48903703	1701
        chr22	48903703	49824534	1702
        chr22	49824534	51243298	1703
        
  If the last column is not available the first 3 columns will be concatenated to become the loci identifier. 

  Note that default data-base for loci can be, naturally, LD blocks. For example European genomes are typically divided into 1703 LD chunks (exclude X chromosome).


- Prior inclusion probability, for example:

      1st columns format “chr:bp:ref:alt”，2nd is prior

        1:1847856:T:G  1.4413e-04
        1:1847979:T:C  7.3716e-05
        1:1848109:C:G  1.4413e-04
        1:1848160:A:G  1.4413e-04
        1:1848734:A:G  7.3716e-05
        
  This is the default format from the enrichment analysis tool called `torus`.
  
- Genotype data reference panel (or panels if by chromosome), in VCF format. Ideally this is the genotype data used to generate the summary statistics; but external reference panel can also be used. 
A popular choice is [1000 Genomes (data download)](http://hgdownload.cse.ucsc.edu/gbdb/hg19/1000Genomes/phase3) for genotypes of 
[different population (data download)](ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel).
    - There are 503 Europeans (`EUR`) in 1000 Genomes data.

## Pitfalls in data mungling with external reference panel

1. Genomic coordinate can be zero- or one-based. That means SNP positions can start from **start from 0**, instead of 1. This is indeed the case for 1000 Genomes data. In order to be consistent with one-based summary statistics, we need to **add 1** to all 1000 genome SNPs position.
2. Reference and alternative alleles may mismatch between summary statistics and the reference panel. If it is a simple ref / alt flip we need to also flip the coding of genotypes or sign of summary statistics to adjust for the effect size direction. If it is strand flip we need to convert summary statistics and genotype data to use the same strand first and take from there. 
3. When strand flip is involved, cases such as A/T and C/G are no longer identifiable -- whether it be strand flip or ref / alt flip. We will have to remove these locus (having A/T or C/G genotypes) from analysis.

## Analysis steps

1. Denote summary statistics matrix in the specific loci as $S_1$, and annotation (prior) matrix as $A_1$. The number of row in these two matrices is the number of SNPs in summary statistics of the study of interest.
2. Extract corresponding genotype from reference panel in VCF format for this loci. Denote this genotype matrix as $G_1$. Rows are SNPs, columns are population genotypes. 
    - Genotype coding: we use numeric coding 0, 1 and 2 indicating the number of "non-reference allele". In other words we have the following "mapping rule":
        ```
        0|0  ->  0
        1|0  ->  1
        1|1  ->  2
        2|0  ->  1
        2|1  ->  2
        2|2  ->  2
        ```
    - Non-variant sites (lines having identical genotypes for everybody) will be removed.
    
3. Find overlapped SNPs of $S_1$ and $G_1$, then extract new genotype matrix from $G_1$ excluding non-overlaps, denote as $G_2$, and new summary statistics matrix from $S_1$ excluding non-overlaps, denote as $S_2$.
4. Compare $G_2$ and $S_2$'s reference and alternative allele, flip coding as necessary, generating new summay statistic matrix $S_3$ and new genotype matrix $G_3$. There could be several situations as follows:

    - completely identical;
    - Not identical, but identical after switching ref and alt in $S_2$: add opposite sign for z score and beta; does not apply to `A/T`, `T/A`, `G/C` or `C/G` for `ref/alt`;
    - Not identical, but identical after strand flip ref and alt in $S_2$: keep the sign of z score and beta; does not apply to `A/T`, `T/A`, `G/C` or `C/G` for `ref/alt`;
    - Not identical, but identical after strand flip ref and alt then swith their positions: add opposite sign for z score and beta; only apply to `A/G`, `G/A`, `A/C`, `C/A`, `T/C`, `C/T`, `T/G` and `G/T`.
    - Not identical, `A/T`, `T/A`, `G/C` or `C/G` for `ref/alt`: consider this situation at last; if flip strand has applied in this LD block, then flip strand and keep the sign of z score and beta; if not, switch ref and alt of $S_2$, add opposite sign for z score and beta.
    - Not identical after previous 5 substeps: drop.
5. Calculate row correlation matrix of $G_3$, denote as $R$. The number of rows and columns of $R$ is the number of SNPs in $G_3$.
6. Obtain overlapped SNPs for $S_3$ and $A_1$ and use overlapped SNPs to generate new annotation/prior $A_2$.

Notice that if genotype data used to generate the summary statistics is available as reference panel, there should not be a need for 4. But it is a good double-check anyways to do 4 -- we therefore provide an option to specify whether or not we expect step 4 is unnecessary; and if so, throw an error when we found discrepency in 4, instead of trying to fix those. 

## Outputs
- $S_3$: the number of rows of $S_3$ is the same with $A_2$. It can be 4 columns, `chr:pos  beta  se  ID`, or 3 columns, `chr:pos z ID`.
- $R$: correlation matrix, #SNPs $\times$ #SNPs of $G_3$.
- $A_2$: adjusted annotation/prior information, the number of rows is the same with $S_3$.

In [1]:
%cd /data/fine_mapping_data/
this_nb = 'summary_statistics_wrangler.ipynb'

/data/fine_mapping_data

In [2]:
[global]
# Current work directory, where output will be written to
parameter: cwd = path(".")
# Reference panel, either VCF file or a manifest file each line is "chrom number <white space> VCF file name"
# the manifest file have to be in the same folder as the VCF files it refers to.
parameter: reference = path()
# Loci file
parameter: loci = path()
# summary statistics
parameter: ss_data = path()
# Use --no-strand-flip to set it to false
# if you are sure there is no strand_flip involved
parameter: strand_flip = True
# Use --no-ref-flip to set it to false
# if you are sure there is no reference / alternative mismatch involved
parameter: ref_flip = True

# Decide whether or not reference panel matches summary statistics
if not ref_flip and not strand_flip:
    strictly_match = True
else:
    strictly_match = False

# Check if files exist
fail_if(not reference.is_file(), msg = 'Please specify valid path for --reference')
fail_if(not loci.is_file(), msg = 'Please specify valid path for --loci')
fail_if(not ss_data.is_file(), msg = 'Please specify valid path for --ss-data')

### Preparing external reference genotypes

For 1000 genomes data, using `integrated_call_samples_v3.20130502.ALL.panel.txt` file to extract sample ID for given population (eg `EUR`) and subset the data. We only need to run it once.

If loci manifest is provided it generate both a new loci manifest file and the files it contains.

In [1]:
# Get information about a specified race
[prepare_1KG_reference]
depends: executable("bcftools"), executable("tabix")
# Race identifier in 1000 genomes
parameter: race = "EUR"
# Path to `integrated_call_samples_v3.20130502.ALL.panel.txt`
parameter: panel_meta = path()
stop_if(not panel_meta.is_file(), msg = 'Please specify valid path for --panel-meta')

if str(reference).endswith('vcf.gz'):
    chroms = ['0']
    genotypes = [reference]
else:
    manifest = [x.strip().split() for x in open(f'{reference:a}').readlines()]
    meta = [x[0] for x in manifest]
    genotypes = [x[1] for x in manifest]

input: sos_groups(genotypes, by = 1).set_each("chrom", chroms), concurrent = True
output: f"{_input:nn}.{race}.vcf.gz"
bash: expand = True, workdir = cwd
    grep -w "{race}" {panel_meta} | cut -f 1 > {_output:nn}_extracted.txt
    bcftools view -S {_output:nn}_extracted.txt {_input} -Oz > {_output}
    tabix -p vcf {_output}
    rm {_output:nn}_extracted.txt

if not str(reference).endswith('vcf.gz'):
    with open(f"{reference:n}.{race}.{reference:x}", 'w') as f:
        for item in _output:
            f.write(f'{item.get("chrom")} {item}')

Example usage:

In [4]:
sos run {this_nb} prepare_1KG_reference

### Obtain reference genotypes for given loci: matrix `𝐺1`

In [5]:
# Get genotype matrix
[default_1 (get genotype)]
# when set to N (typically 0, 1 or -1) the genomic position
# in the panel will be adjusted by N, ie. position = position + N
# This is useful when summary stats and reference panel have different coordinates (off by 1 position)
parameter: adjust_panel_position = 0
depends: Py_Module('cyvcf2')

import pandas as pd
chunks = [x.tolist() for idx, x in pd.read_table(f'{loci:a}', header = None, sep = '\s+').iterrows() if not x[0].startswith('#')]
if str(reference).endswith('vcf.gz'):
    genotype = reference
else:
    genotype = dict([tuple(x.strip().split()) for x in open(f'{reference:a}').readlines()])
    for k in genotype:
        genotype[k] = os.path.join(f'{reference:ad}', genotype[k])

input: group_by = 1, for_each = 'chunks', concurrent = True
output: sos_targets(f"{ss_data:n}.{_chunks[-1] if len(_chunks) > 3 else '%s_%s_%s' % (_chunks[0], _chunks[1], _chunks[2])}.pkl").set("chunks", chunks)
python3: expand = "${ }", workdir = cwd
    from cyvcf2 import VCF
    import pandas as pd
    chromosome = '${_chunks[0].replace("chr","")}'
    panel = ${path(genotype[_chunks[0].replace("chr","")] if isinstance(genotype, dict) else genotype):ar}
    # loci region
    queryid =  chromosome + ":" + "${_chunks[1]}" + "-" + "${_chunks[2]}"
    off_set = ${adjust_panel_position}
    # scan VCF chunk
    vcf = VCF(panel, gts012=False)
    res = []
    ## Attention: 1000 Genome position start from 0, not 1! So need to +1 for variant.start
    for variant in vcf(queryid):
        for i in range(len(variant.ALT)):
            line = [f'{variant.CHROM}:{variant.start+off_set}', 
                    variant.REF, variant.ALT[i]] + \
                    [x[:-1].count(i+1) for x in variant.genotypes]
            if len(set(line[3:])) == 1:
                # remove non-variant site in reference panel
                continue
            res.append(line)
    gt = pd.DataFrame(res, columns = ['ID', 'ref', 'alt'] + vcf.samples)
    gt.to_pickle(${_output:r})

### Obtain summary statistics and the matching genotype correlation for given loci

Obtain genotype matrix 𝐺2, summary statistics 𝑆2, compare ref/alt in 𝑆2 and 𝐺2 => 𝑆3 and 𝐺3, calculate 𝑅=row_corr(𝐺3)

In [6]:
[default_2 (get summary stats)]
input: group_by = 1, concurrent = True
output: summary_stats = f"{_input:n}.summary_stats.txt", ld_matrix = f"{_input:n}.LD.txt"
python3: expand="${ }", stdout = f'{_input:n}.allele_flip.log', workdir = cwd
    import pandas as pd, numpy as np
    import datetime, sys

    def exit_if_empty(x):
        if x.empty:
            open("${_output['summary_stats']}", 'w').close()
            open("${_output['ld_matrix']}", 'w').close()
            sys.exit(0)

    print(datetime.datetime.now(),"\n")
    locus = ${step_input.get("chunks")[_index]}
    S1 = pd.read_table(${ss_data:ar}, compression='gzip', header = 0)
    S1 = S1[(S1["chr"] == locus[0]) & (S1["bp"] >= int(locus[1])) & (S1["bp"] < int(locus[2]))]
    exit_if_empty(S1)
    print("SAMPLE: %s\tLENGTH:%d\n" % ("S1",len(S1)))
    S1["chr"] = S1["chr"].str.replace("chr", "")
    S1["ID"] = S1["chr"] + ":" + S1["bp"].map(str)
    G1 = pd.read_pickle("${_input}")
    print("SAMPLE: %s\tLENGTH:%d\n" % ("G1",len(G1)))
    # S2
    S2 = S1[S1["ID"].isin(G1["ID"])]
    exit_if_empty(S2)
    #S2["ID"] = S2["ID"].fillna
    print("SAMPLE: %s\tLENGTH:%d\n" % ("S2",len(S2)))
    # 𝐺2
    G2 = G1[G1["ID"].isin(S1["ID"])].copy()
    exit_if_empty(G2)
    print("SAMPLE: %s\tLENGTH:%d\n" % ("G2",len(G2)))
    # 𝑆2 and 𝐺2 : reference and alternative
    # at ta cg gc
    def gt(s1,s2,s3,s4):
        if (s1+s2 == "AT" and s3+s4 == "TA") or (s1+s2 == "GC" and s3+s4 == "CG" ) or (s1+s2 == "TA" and s3+s4 == "AT") or (s1+s2 == "CG" and s3+s4 == "GC"):
            return ${0 if strand_flip else 1}
        else:
            return 1
    # flip
    def atcg(inp):
        if inp == "A":
            return "T"
        elif inp == "T":
            return "A"
        elif inp == "G":
            return "C"
        elif inp == "C":
            return "G"
    # S3 
    FLIPD = []
    # equal
    EQUAL = 0
    # flipped by strand 
    FLIPNUM = 0
    # flipped by reference and alternative
    REFALTNUM = 0 
    
    # keep at/ta/cg/gc line numbers
    at_cg_id = []


    for i in range(len(S2)):
        line = list(S2.iloc[i,:2])
        old_id = S2.iloc[i,0] + ":" + str(S2.iloc[i,1]) + ":" + S2.iloc[i,2] + ":" + S2.iloc[i,3]
        if set([S2.iloc[i,2],S2.iloc[i,3]]) != set([G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0],G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0]]):
            print (S2.iloc[i,1],S2.iloc[i,2],S2.iloc[i,3],S2.iloc[i,-1], G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0],G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0])
        # not at/ta/gc/cg
        if gt(S2.iloc[i,2],S2.iloc[i,3],G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0],G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0])==1:
            if G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0] == S2.iloc[i,2] and G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0] == S2.iloc[i,3]:
                EQUAL+=1
                line=S2.iloc[i,:-1].tolist() + [old_id]
                FLIPD.append(line)
            elif G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0] == S2.iloc[i,2] and G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0] == S2.iloc[i,3]:
                REFALTNUM+=1
                line.extend([S2.iloc[i,3],S2.iloc[i,2],0-S2.iloc[i,4],0-S2.iloc[i,5],S2.iloc[i,6], old_id])
                FLIPD.append(line)
            elif atcg(S2.iloc[i,2]) == G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0] and atcg(S2.iloc[i,3]) == G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0]:
                FLIPNUM+=1
                line.extend([atcg(S2.iloc[i,2]),atcg(S2.iloc[i,3]),S2.iloc[i,4],S2.iloc[i,5],S2.iloc[i,6], old_id])
                FLIPD.append(line)
            elif atcg(S2.iloc[i,2]) == G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0] and atcg(S2.iloc[i,3]) == G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0]:
                FLIPNUM+=1
                tmp = S2.iloc[i,2]
                line.extend([atcg(S2.iloc[i,3]),atcg(tmp),0-S2.iloc[i,4],0-S2.iloc[i,5],S2.iloc[i,6], old_id])
                FLIPD.append(line)
        # at/ta/cg/gc only do ref/alt flip
        elif gt(S2.iloc[i,2],S2.iloc[i,3],G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0],G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0])==0:
            if G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0] == S2.iloc[i,2] and G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0] == S2.iloc[i,3]:
                EQUAL+=1
                line=S2.iloc[i,:-1].tolist() + [old_id]
                FLIPD.append(line)
                at_cg_id.append(len(FLIPD) - 1)
            elif G2[G2["ID"]==S2.iloc[i,-1]]["alt"].iloc[0] == S2.iloc[i,2] and G2[G2["ID"]==S2.iloc[i,-1]]["ref"].iloc[0] == S2.iloc[i,3]:
                REFALTNUM+=1
                line.extend([S2.iloc[i,3],S2.iloc[i,2],0-S2.iloc[i,4],0-S2.iloc[i,5],S2.iloc[i,6], old_id])
                FLIPD.append(line)
                at_cg_id.append(len(FLIPD) - 1)
    if ${strictly_match} and (FLIPNUM > 0 or REFALTNUM > 0):
        raise ValueError(f"Strict panel match failed for locus {locus}.")
    #
    print("equal:%s\tflipped by strand:%d\tflipped by reference and alternative:%d\n" % (EQUAL,FLIPNUM,REFALTNUM))
    if FLIPNUM > 0:
        # flips involved, we need to remove at/ta/cg/gc
        FLIPD = [i for j, i in enumerate(FLIPD) if j not in at_cg_id]
    S3 = pd.DataFrame(FLIPD, columns=["chr","bp","ref","alt","z", "beta", "se", "original_ID"])
    exit_if_empty(S3)
    S3["ID"] = S3["chr"] +":"+S3["bp"].map(str) + ":" + S3["ref"] + ":" + S3["alt"]
    G2["ID"] = G2[['ID', 'ref', 'alt']].apply(lambda x: ':'.join(x), axis=1)
    # 𝐺3
    G3 = G2[G2["ID"].isin(S3["ID"])]
    assert G3.shape[0] == S3.shape[0]
    # S3 OUT
    if "z" in S3.columns and "beta" in S3.columns and "se" in S3.columns:
        S3 = S3.drop(columns = ["z"])
    # recover original ID
    print(S3)
    S3["ID"] = S3["original_ID"]
    S3 = S3.sort_values(by = ["chr", "bp"]).drop(columns = ["chr", "bp", "ref", "alt", "original_ID"])
    columns = S3.columns.tolist()
    columns.insert(0, columns.pop(columns.index('ID')))
    S3[columns].to_csv("${_output['summary_stats']}",sep="\t",index=False)
    print("SAMPLE: %s\tLENGTH:%d\n" % ("S3",len(S3)))
    # 𝑅=row_corr(𝐺3) OUT 
    np.savetxt("${_output['ld_matrix']}", np.corrcoef(G3.drop(columns=['ID', "ref","alt"]).values), fmt = '%.5f')
#_input.zap()

### Obtain annotation $A_2$

In [7]:
[default_3 (get annotations)]
parameter: anno = path()
stop_if(not anno.is_file(), msg = 'Quit because annotation file is not found')
input: [x for x in output_from(-1) if os.stat(x).st_size > 0], group_by = 2, concurrent = True
output: f"{_input[0]:nn}.{anno:bn}.txt"
python3: expand="${ }", workdir = cwd
    import pandas as pd
    A1 = pd.read_table("${anno}", compression='gzip', sep="\s+", header=None)
    A1.columns = ["ID","VALUE"]
    S3 = pd.read_table("${_input[0]}",sep="\t")
    A2 = A1[A1["ID"].isin(S3.iloc[:,0])][["ID","VALUE"]]
    A2.to_csv("${_output}",sep="\t",index=False,header=None)

### Results of LD chunk 655

### Flip summary

In [ ]:
%preview /data/chr12_4/Summary_statistics/chunk_4.flip_summary -n --limit 20

The number of SNPs in 

- S1: 8286510
- G1: 35634
- S2: 112
- G2: 112
- S3: 67

The number of identical SNPs in S2 and G2 is 10; strand flip 50 SNPs; switch ref/alt 7 SNPs.

### Summary statistics matrix S3

In [ ]:
%preview Summary_statistics/chunk_1696.matched_ss.txt -n

### Annotation/prior matrix A2

In [ ]:
%preview Summary_statistics/chunk_1696.annotation.txt -n

### 𝑅: LD matrix

In [ ]:
summary(read.table('Summary_statistics/chunk_1696.LD.txt'))